In [12]:
# ============================================================
# KMU Comovement - 강화 베이스라인
# - rule-based (corr + stability)
# - LightGBM (main) + CatBoost (secondary)
# - log-target + 앙상블
# ============================================================

import math
import numpy as np
import pandas as pd
from tqdm import tqdm

# ------------------------------------------------------------
# 1. 데이터 로드
# ------------------------------------------------------------
train = pd.read_csv("./train.csv")
sub_template = pd.read_csv("./sample_submission.csv")

print("train.shape:", train.shape)
print("submission template shape:", sub_template.shape)

# ------------------------------------------------------------
# 2. 월별 집계 + 피벗 (item_id × ym)
# ------------------------------------------------------------
# (1) item_id × year × month 합산
monthly = (
    train
    .groupby(["item_id", "year", "month"], as_index=False)["value"]
    .sum()
)

# (2) year, month → ym (datetime 월 단위)
monthly["ym"] = pd.to_datetime(
    monthly["year"].astype(str) + "-" + monthly["month"].astype(str).str.zfill(2)
)

# (3) item_id × ym 피벗
pivot = (
    monthly
    .pivot(index="item_id", columns="ym", values="value")
    .fillna(0.0)
)

print("pivot shape:", pivot.shape)
display(pivot.head())

# ------------------------------------------------------------
# 3. 공행성 쌍 탐색 - corr + stability rule-based
# ------------------------------------------------------------
def safe_corr(x, y):
    """
    표준편차 0 등 예외 케이스 안전 처리 상관계수
    """
    x = np.asarray(x, float)
    y = np.asarray(y, float)
    if x.size == 0 or y.size == 0:
        return 0.0
    if np.std(x) == 0 or np.std(y) == 0:
        return 0.0
    return float(np.corrcoef(x, y)[0, 1])


def stability_score(a_seg, b_seg):
    """
    간단 stability 지표:
    - 월별 변화량(Δ)의 방향이 얼마나 자주 일치하는지 비율
    """
    a_seg = np.asarray(a_seg, float)
    b_seg = np.asarray(b_seg, float)

    if len(a_seg) <= 2:
        return 0.0

    da = np.diff(a_seg)
    db = np.diff(b_seg)

    sign_a = np.sign(da)
    sign_b = np.sign(db)

    mask = (sign_a != 0) | (sign_b != 0)
    if mask.sum() == 0:
        return 0.0

    agree = (sign_a[mask] == sign_b[mask]).mean()
    return float(agree)


def find_comovement_pairs(
    pivot,
    max_lag: int = 6,
    min_nonzero: int = 6,
    corr_threshold: float = 0.3,
    stability_threshold: float = 0.55,
):
    """
    corr + stability 둘 다 만족하는 (A,B) 쌍만 공행성 쌍으로 채택
    """
    items = pivot.index.to_list()
    months = pivot.columns.to_list()
    n_months = len(months)

    results = []

    for leader in tqdm(items, desc="find_comovement_pairs - leader"):
        x = pivot.loc[leader].values.astype(float)
        if np.count_nonzero(x) < min_nonzero:
            continue

        for follower in items:
            if follower == leader:
                continue

            y = pivot.loc[follower].values.astype(float)
            if np.count_nonzero(y) < min_nonzero:
                continue

            best_corr = 0.0
            best_lag = None
            best_stab = 0.0

            for lag in range(1, max_lag + 1):
                t_start = lag
                t_end = n_months

                a_seg = x[t_start - lag : t_end - lag]
                b_seg = y[t_start:t_end]

                if len(a_seg) < 3:
                    continue

                c = safe_corr(a_seg, b_seg)
                s = stability_score(a_seg, b_seg)

                # corr 중심으로 선택 (stability는 기록만)
                if abs(c) > abs(best_corr):
                    best_corr = c
                    best_lag = lag
                    best_stab = s

            if (
                best_lag is not None
                and abs(best_corr) >= corr_threshold
                and best_stab >= stability_threshold
            ):
                results.append(
                    {
                        "leading_item_id": leader,
                        "following_item_id": follower,
                        "best_lag": int(best_lag),
                        "max_corr": float(best_corr),
                        "stability": float(best_stab),
                    }
                )

    pairs = pd.DataFrame(results)
    return pairs


# 실제 실행
pairs = find_comovement_pairs(
    pivot,
    max_lag=6,
    min_nonzero=6,
    corr_threshold=0.3,      # 튜닝 포인트 1
    stability_threshold=0.63 # 튜닝 포인트 2
)

print("탐색된 공행성 쌍 수:", len(pairs))
display(pairs.head())

# ------------------------------------------------------------
# 4. 학습 데이터 생성 (self-supervised)
# ------------------------------------------------------------
def build_training_data(pivot, pairs):
    """
    공행성쌍 + 시계열에서 (X, y) 만들기
    - target: 다음달 b_{t+1} 의 log1p 값
    """
    months = pivot.columns.to_list()
    n_months = len(months)
    rows = []

    for row in tqdm(pairs.itertuples(index=False), total=len(pairs), desc="build_training_data"):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)
        stab = float(row.stability)

        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        # t-1, t, t+1, t-lag 모두 존재해야 함
        t_min = max(lag, 1)
        t_max = n_months - 1  # t+1까지 이용

        for t in range(t_min, t_max):
            b_t = b_series[t]
            b_t_1 = b_series[t - 1]
            a_t_lag = a_series[t - lag]
            y = b_series[t + 1]

            rows.append(
                {
                    "leading_item_id": leader,
                    "following_item_id": follower,
                    "t_idx": t,
                    "b_t": b_t,
                    "b_t_1": b_t_1,
                    "a_t_lag": a_t_lag,
                    "max_corr": corr,
                    "best_lag": lag,
                    "stability": stab,
                    "log_b_t": math.log1p(b_t),
                    "log_b_t_1": math.log1p(b_t_1),
                    "log_a_t_lag": math.log1p(a_t_lag),
                    "delta_b": b_t - b_t_1,
                    "rel_delta_b": (b_t - b_t_1) / (b_t_1 + 1.0),
                    "target": y,
                    "log_target": math.log1p(y),
                }
            )

    df = pd.DataFrame(rows)
    return df


df_train_model = build_training_data(pivot, pairs)
print("df_train_model shape:", df_train_model.shape)
display(df_train_model.head())

# ------------------------------------------------------------
# 5. LightGBM (main) + CatBoost (secondary) 학습
# ------------------------------------------------------------
feature_cols = [
    "b_t",
    "b_t_1",
    "a_t_lag",
    "max_corr",
    "best_lag",
    "stability",
    "log_b_t",
    "log_b_t_1",
    "log_a_t_lag",
    "delta_b",
    "rel_delta_b",
]

train_X = df_train_model[feature_cols]
train_y = df_train_model["log_target"].values  # log1p 타깃

models = {}

# 5-1. LightGBM
try:
    import lightgbm as lgb

    print("Training LightGBMRegressor...")
    lgb_model = lgb.LGBMRegressor(
        n_estimators=1200,
        learning_rate=0.03,
        max_depth=-1,
        num_leaves=63,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=-1,
    )
    lgb_model.fit(train_X, train_y)
    models["lgb"] = lgb_model
except ImportError:
    print("LightGBM이 설치되어 있지 않습니다. !pip install lightgbm 후 다시 시도하세요.")

# 5-2. CatBoost
try:
    from catboost import CatBoostRegressor

    print("Training CatBoostRegressor...")
    cat_model = CatBoostRegressor(
        depth=8,
        learning_rate=0.05,
        n_estimators=1000,
        loss_function="RMSE",
        random_seed=42,
        verbose=False,
    )
    cat_model.fit(train_X, train_y)
    models["cat"] = cat_model
except ImportError:
    print("CatBoost가 설치되어 있지 않습니다. !pip install catboost 후 다시 시도하세요.")


# 5-3. log-space 앙상블 함수
def ensemble_predict_log(X):
    """
    등록된 모델들(LGB, CatBoost 등)의 log 예측값을 평균
    """
    preds = []
    for name, m in models.items():
        p = m.predict(X)
        preds.append(p)

    if len(preds) == 0:
        # 방어 코드: 모델이 하나도 없으면 0
        return np.zeros(X.shape[0])

    preds = np.vstack(preds)  # (n_models, n_samples)
    avg_log = preds.mean(axis=0)
    return avg_log

# ------------------------------------------------------------
# 6. 추론용: (A,B) pair별 best_lag, corr, stability 계산
# ------------------------------------------------------------
def compute_best_lag_corr_stab_for_pair(pivot, leader, follower, max_lag=6):
    """
    inference용 (A,B) 쌍에 대해 best_lag, max_corr, stability 계산
    """
    x = pivot.loc[leader].values.astype(float)
    y = pivot.loc[follower].values.astype(float)
    n_months = x.shape[0]

    best_corr = 0.0
    best_lag = 1
    best_stab = 0.0

    for lag in range(1, max_lag + 1):
        t_start = lag
        t_end = n_months

        a_seg = x[t_start - lag : t_end - lag]
        b_seg = y[t_start:t_end]

        if len(a_seg) < 3:
            continue

        c = safe_corr(a_seg, b_seg)
        s = stability_score(a_seg, b_seg)

        if abs(c) > abs(best_corr):
            best_corr = c
            best_lag = lag
            best_stab = s

    return best_lag, best_corr, best_stab

# ------------------------------------------------------------
# 7. 제출 파일 전체에 대해 앙상블 예측
# ------------------------------------------------------------
def predict_for_submission_only_pairs(pivot, pairs, max_lag=6):
    """
    공행성 있다고 rule-based로 선정된 pairs 에 대해서만
    다음달 무역량을 예측해서 제출용 DataFrame 생성
    """
    months = pivot.columns.to_list()
    n_months = len(months)

    t_last = n_months - 1   # 마지막 달 index
    t_prev = n_months - 2   # 마지막-1 달 index

    preds = []

    for row in tqdm(pairs.itertuples(index=False), total=len(pairs), desc="predict_only_pairs"):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)
        stab = float(row.stability)

        # 혹시라도 피벗에 없는 아이템이면 스킵 (또는 0 예측)
        if leader not in pivot.index or follower not in pivot.index:
            continue

        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        if t_last - lag < 0:
            # lag가 너무 크면 그냥 마지막 값으로 대체
            y_pred = float(b_series[t_last])
        else:
            b_t = b_series[t_last]
            b_t_1 = b_series[t_prev]
            a_t_lag = a_series[t_last - lag]

            feat = {
                "b_t": b_t,
                "b_t_1": b_t_1,
                "a_t_lag": a_t_lag,
                "max_corr": corr,
                "best_lag": lag,
                "stability": stab,
                "log_b_t": math.log1p(b_t),
                "log_b_t_1": math.log1p(b_t_1),
                "log_a_t_lag": math.log1p(a_t_lag),
                "delta_b": b_t - b_t_1,
                "rel_delta_b": (b_t - b_t_1) / (b_t_1 + 1.0),
            }

            # 🔴 여기 DataFrame으로 만들어줘야 LGBM 경고도 안 뜸
            X_test_df = pd.DataFrame([feat])[feature_cols]
            log_pred = float(ensemble_predict_log(X_test_df)[0])
            y_pred = math.expm1(log_pred)

        y_pred = max(0.0, y_pred)
        y_pred = int(round(y_pred))

        preds.append(
            {
                "leading_item_id": leader,
                "following_item_id": follower,
                "value": y_pred,
            }
        )

    df_pred = pd.DataFrame(preds)
    return df_pred


# 실제 예측 & 저장
submission = predict_for_submission_only_pairs(pivot, pairs, max_lag=6)
print("제출용 행 개수:", len(submission))
submission.head()

submission.to_csv("submission_only_comovement_pairs.csv", index=False)



train.shape: (10836, 9)
submission template shape: (9900, 3)
pivot shape: (100, 43)


ym,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,...,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01
item_id,,,,,,,,,,,,,,,,,,,,,
AANGBULD,14276.0,52347.0,53549.0,0.0,26997.0,84489.0,0.0,0.0,0.0,0.0,...,428725.0,144248.0,26507.0,25691.0,25805.0,0.0,38441.0,0.0,441275.0,533478.0
AHMDUILJ,242705.0,120847.0,197317.0,126142.0,71730.0,149138.0,186617.0,169995.0,140547.0,89292.0,...,123085.0,143451.0,78649.0,125098.0,80404.0,157401.0,115509.0,127473.0,89479.0,101317.0
ANWUJOKX,0.0,0.0,0.0,63580.0,81670.0,26424.0,8470.0,0.0,0.0,80475.0,...,0.0,0.0,0.0,27980.0,0.0,0.0,0.0,0.0,0.0,0.0
APQGTRMF,383999.0,512813.0,217064.0,470398.0,539873.0,582317.0,759980.0,216019.0,537693.0,205326.0,...,683581.0,2147.0,0.0,25013.0,77.0,20741.0,2403.0,3543.0,32430.0,40608.0
ATLDMDBO,143097177.0,103568323.0,118403737.0,121873741.0,115024617.0,65716075.0,146216818.0,97552978.0,72341427.0,87454167.0,...,60276050.0,30160198.0,42613728.0,64451013.0,38667429.0,29354408.0,42450439.0,37136720.0,32181798.0,57090235.0


find_comovement_pairs - leader: 100%|██████████| 100/100 [00:13<00:00,  7.33it/s]

탐색된 공행성 쌍 수: 394


,leading_item_id,following_item_id,best_lag,max_corr,stability
0,AANGBULD,DDEXPPXU,2,0.383169,0.650000
1,AHMDUILJ,ATLDMDBO,4,0.483281,0.657895
2,AHMDUILJ,BEZYMBBT,4,0.319341,0.684211
3,AHMDUILJ,BUZIIBYG,4,0.461739,0.657895
4,AHMDUILJ,DUCMGGNW,1,0.450955,0.731707


...

df_train_model shape: (15093, 16)


,leading_item_id,following_item_id,t_idx,b_t,b_t_1,a_t_lag,max_corr,best_lag,stability,log_b_t,log_b_t_1,log_a_t_lag,delta_b,rel_delta_b,target,log_target
0,AANGBULD,DDEXPPXU,2,38475.0,12187.0,14276.0,0.383169,2,0.65,10.557790,9.408207,9.566405,26288.0,2.156876,23209.0,10.052338
1,AANGBULD,DDEXPPXU,3,23209.0,38475.0,52347.0,0.383169,2,0.65,10.052338,10.557790,10.865669,-15266.0,-0.396767,37804.0,10.540197
2,AANGBULD,DDEXPPXU,4,37804.0,23209.0,53549.0,0.383169,2,0.65,10.540197,10.052338,10.888371,14595.0,0.628824,27145.0,10.208985
3,AANGBULD,DDEXPPXU,5,27145.0,37804.0,0.0,0.383169,2,0.65,10.208985,10.540197,0.000000,-10659.0,-0.281947,1210.0,7.099202
4,AANGBULD,DDEXPPXU,6,1210.0,27145.0,26997.0,0.383169,2,0.65,7.099202,10.208985,10.203518,-25935.0,-0.955389,5943.0,8.690138


Training LightGBMRegressor...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001884 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2334
[LightGBM] [Info] Number of data points in the train set: 15093, number of used features: 11
[LightGBM] [Info] Start training from score 12.702178
Training CatBoostRegressor...


predict_only_pairs: 100%|██████████| 394/394 [00:03<00:00, 117.68it/s]

제출용 행 개수: 394
